## Install and Imports

In [1]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [2]:
!pip install transformers
!pip install datasets
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 63.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.5 MB/s eta 0:00:0

In [3]:
import os
from tqdm import tqdm
import numpy as np
import librosa
import torch
import torch.nn as nn
import torch.optim as optim
import torchaudio
import datasets
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import Wav2Vec2ForCTC, AutoFeatureExtractor, Wav2Vec2FeatureExtractor, Wav2Vec2ForSequenceClassification, AdamW, AutoConfig, Wav2Vec2Processor

In [4]:
# Mount google drive to use a persistent directory structure
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/MyDrive/deep_learning/final_proj')

Mounted at /content/drive


## Data loading and processing

In [5]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import load_dataset

# Define the genres and their corresponding folders
genres = ['Hip-Hop', 'Rock', 'Pop', 'Folk', 'Experimental', 'Electronic', 'Instrumental', 'International']
data_path = 'fma_filtered'  # Path to your data folder

# Create a list to store the data
data = []

# Loop over each genre folder
for genre in genres:
    genre_folder = os.path.join(data_path, genre)
    
    # Get the audio files in the genre folder
    audio_files = os.listdir(genre_folder)
    
    # Sample 100 files from each genre
    audio_files = audio_files[:400]
    
    # Add the data to the list
    for audio_file in audio_files:
        audio_path = os.path.join(genre_folder, audio_file)
        data.append({
            'input_values': audio_path,
            'labels': genre
        })

# Convert the list to a DataFrame
df = pd.DataFrame(data)

# Split the data into train and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=101, stratify=df['labels'])

# Save the train and test sets as CSV files
train_df.to_csv('train.csv', index=False)
test_df.to_csv('test.csv', index=False)

# Load the dataset using the Hugging Face datasets library
dataset = load_dataset('csv', data_files={'train': 'train.csv', 'validation': 'test.csv'}, cache_dir=False)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-af35a94804cc8375/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
# Get the train and validation datasets
train_dataset = dataset['train']
eval_dataset = dataset['validation']

# Preprocess the data
def preprocess_function(examples):
    # Perform your preprocessing steps here
    return examples

train_dataset = train_dataset.map(preprocess_function)
eval_dataset = eval_dataset.map(preprocess_function)

In [9]:
# We need to specify the input and output column
input_column = "input_values"
output_column = "labels"

In [10]:
# we need to distinguish the unique labels in our SER dataset
label_list = train_dataset.unique(output_column)
label_list.sort()  # Let's sort it for determinism
num_labels = len(label_list)
print(f"A classification problem with {num_labels} classes:\n {label_list}")

A classification problem with 8 classes:
 ['Electronic', 'Experimental', 'Folk', 'Hip-Hop', 'Instrumental', 'International', 'Pop', 'Rock']


In [11]:
model_name_or_path = "facebook/wav2vec2-base-100k-voxpopuli"
pooling_mode = "mean"

config = AutoConfig.from_pretrained(
    model_name_or_path,
    num_labels=num_labels,
    label2id={label: i for i, label in enumerate(label_list)},
    id2label={i: label for i, label in enumerate(label_list)},
    finetuning_task="wav2vec2_clf",
)
setattr(config, 'pooling_mode', pooling_mode)

In [12]:
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_name_or_path)
target_sampling_rate = feature_extractor.sampling_rate
print(f"The target sampling rate: {target_sampling_rate}")

The target sampling rate: 16000


In [13]:
train_dataset

Dataset({
    features: ['input_values', 'labels'],
    num_rows: 640
})

In [14]:

def label_to_id(label, label_list):
    if len(label_list) > 0:
        return label_list.index(label) if label in label_list else -1
    return label

from datasets import Dataset

def load_preprocessed(example):
    speech_path = os.path.join("preprocessed_tensors_full/", example['input_values'].split("/")[-1] + '.pt')
    if os.path.exists(speech_path):
        speech = torch.load(speech_path).squeeze(0)[:479626]
        example['input_values'] = torch.tensor(speech)  # ensure that 'speech' is a tensor
        example['labels'] = label_to_id(example['labels'], label_list)
        example['file_exists'] = True
    else:
        example['file_exists'] = False
    return example

In [15]:
def check_file_exists(example):
    speech_path = os.path.join("preprocessed_tensors_full/", example['input_values'].split("/")[-1] + '.pt')
    return os.path.exists(speech_path)

# Find the indices of examples that you want to keep
train_indices = [i for i, example in enumerate(train_dataset) if check_file_exists(example)]
eval_indices = [i for i, example in enumerate(eval_dataset) if check_file_exists(example)]

# Select only the examples with those indices
train_dataset = train_dataset.select(train_indices)
eval_dataset = eval_dataset.select(eval_indices)

In [16]:
# Now apply your map function only on the selected examples
# train_dataset = train_dataset.map(load_preprocessed, remove_columns=["file_exists"])
# eval_dataset = eval_dataset.map(load_preprocessed, remove_columns=["file_exists"])

# Now apply your map function only on the selected examples
train_dataset = train_dataset.map(load_preprocessed)
eval_dataset = eval_dataset.map(load_preprocessed)

Map:   0%|          | 0/640 [00:00<?, ? examples/s]

<ipython-input-14-e9c618461d67>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  example['input_values'] = torch.tensor(speech)  # ensure that 'speech' is a tensor


Map:   0%|          | 0/160 [00:00<?, ? examples/s]

In [17]:
train_dataset.set_format(type='torch', columns=['input_values', 'labels'])
eval_dataset.set_format(type='torch', columns=['input_values', 'labels'])

In [18]:
# train_dataset.save_to_disk("train_dataset.hf")
# eval_dataset.save_to_disk("eval_dataset.hf")

In [19]:
train_dataset

Dataset({
    features: ['input_values', 'labels', 'file_exists'],
    num_rows: 640
})

In [20]:
train_dataset[120]['labels']

tensor(5)

## Define Model / Train Set-Up

In [21]:
from datasets import load_dataset, load_metric
metric = load_metric("accuracy")

<ipython-input-21-2081cfe081e3>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [22]:
model_checkpoint = "facebook/wav2vec2-base"
#model_checkpoint = "facebook/wav2vec2-large"
batch_size = 32

In [23]:
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer

num_labels = len(label_list)
model = AutoModelForAudioClassification.from_pretrained(
    model_checkpoint, 
    num_labels=num_labels,
    label2id={label: i for i, label in enumerate(label_list)},
    id2label={i: label for i, label in enumerate(label_list)},
    hidden_dropout = 0.1,
    activation_dropout = 0.1,
    attention_dropout = 0.1,
    feat_proj_dropout = 0.1,
    feat_quantizer_dropout = 0.0,
    final_dropout = 0.1
)

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:380: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2ForSequenceClassification: ['quantizer.weight_proj.bias', 'quantizer.codevectors', 'project_q.bias', 'quantizer.weight_proj.weight', 'project_hid.bias', 'project_hid.weight', 'project_q.weight']
- This IS expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['projector.weight', 'classifier.weight', 'classifi

In [24]:
# class CustomTrainer(Trainer):
#     def log(self, logs: Dict[str, Any], iterator: Optional[tqdm] = None) -> None:
#         if self.state.is_local_process_zero:
#             self._total_flos += self.state.total_flos
#         logs["total_flos"] = self._total_flos

#         if self.epoch_logging is not None:
#             logs["epoch"] = self.epoch_logging

#         # Add training accuracy to logs
#         if 'loss' in logs:
#             predictions, labels = self.predict(self.train_dataset)
#             predictions = np.argmax(predictions, axis=1)
#             train_acc = np.sum(predictions == labels) / labels.shape[0]
#             logs['train_accuracy'] = train_acc

#         self._log(logs)

In [26]:
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    output_dir="wav2vec2-base-ek-d",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=6,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=6,
    num_train_epochs=20,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

In [27]:
import numpy as np

def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [28]:
# from transformers import TrainerCallback
# from copy import deepcopy

# class CustomCallback(TrainerCallback):
    
#     def __init__(self, trainer) -> None:
#         super().__init__()
#         self._trainer = trainer
    
#     def on_epoch_end(self, args, state, control, **kwargs):
#         if control.should_evaluate:
#             control_copy = deepcopy(control)
#             self._trainer.evaluate(eval_dataset=self._trainer.train_dataset, metric_key_prefix="train")
#             return control_copy

In [29]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics
)

In [30]:
# trainer.add_callback(CustomCallback(trainer)) 

## Train and Evaluate

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
0,2.076000,2.051767,0.312500
1,1.971300,1.917079,0.343750
2,1.808500,1.777270,0.437500
4,1.770800,1.727639,0.406250
4,1.607800,1.666650,0.450000
5,1.533500,1.696872,0.393750
6,1.432700,1.602646,0.481250
8,1.411100,1.554166,0.500000
8,1.221900,1.507935,0.518750
9,1.121100,1.437137,0.543750


In [ ]:
# checkpoint = torch.load("wav2vec2-base-ek-2/checkpoint-1440")
# model.load_state_dict(checkpoint['state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer'])
#     return model, optimizer, checkpoint['epoch']

In [ ]:
trainer.save_model("wav2vec2-base-ek-d")

In [ ]:
evaluation_metrics = trainer.evaluate()

In [ ]:
import json
from sklearn.metrics import confusion_matrix
import numpy as np

# Get evaluation metrics
eval_metrics = trainer.evaluate()

# Save evaluation metrics
with open("metrics/evaluation_metrics_base_d.json", "w") as outfile:
    json.dump(eval_metrics, outfile)

In [ ]:
# Get the prediction and labels
predictions = trainer.predict(eval_dataset).predictions
predictions = np.argmax(predictions, axis=1)
labels = eval_dataset["labels"]

In [ ]:
# Print classification report
report = classification_report(labels, predictions)
print(report)
with open('metrics/classification_report_base_d.txt', 'w') as f:
    f.write(report)

# Compute the confusion matrix
cm = confusion_matrix(labels, predictions)
# Save confusion matrix
np.savetxt("metrics/confusion_matrix_base_d.csv", cm, delimiter=",")